In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./your-model")
embedding = model.encode(chapter_text)

sample = {
  "chapter_id": "ch03_assembly",
  "text": "이 장에서는 조립 순서를 다룬다. 첫 번째 단계는...",
  "image_urls": [
    "https://internal.corp/images/ch03_step1.png",
    "https://internal.corp/images/ch03_step2.png"
  ]
}

In [ ]:
from pymilvus import FieldSchema, CollectionSchema, Collection, DataType

fields = [
    FieldSchema(name="chapter_id", dtype=DataType.VARCHAR, is_primary=True, max_length=100),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=len(embedding)),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2000),
    FieldSchema(name="image_urls", dtype=DataType.VARCHAR, max_length=1000)
]

schema = CollectionSchema(fields, description="Manual chapters with text and images")
collection = Collection(name="manual_chapters", schema=schema)


In [ ]:
query_embedding = model.encode(user_query)
collection.load()

results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "cosine", "params": {"nprobe": 10}},
    limit=3,
    output_fields=["chapter_id", "text", "image_urls"]
)
